In [29]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import classification_report

In [30]:
train = pd.read_csv(r"D:\pythonC\Brainwork Assignments\twitter_sentiment\Data\processed\train.csv")
test  = pd.read_csv(r"D:\pythonC\Brainwork Assignments\twitter_sentiment\Data\processed\test.csv")


In [31]:
train.dropna(inplace=True)
test.dropna(inplace=True)

In [32]:
X_train = train["cleaned_text"].values
y_train = train["target"].values
X_test  = test["cleaned_text"].values
y_test  = test["target"].values


In [33]:
vocab_size = 5000
max_len = 50

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

In [22]:
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

y_train shape: (1246623,)
y_test shape: (155830,)


In [18]:
print("Vocab size:", vocab_size)
print("Max sequence length:", max_len)

Vocab size: 5000
Max sequence length: 50


In [34]:
model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=64, input_length=max_len),
        LSTM(64, dropout=0.2, recurrent_dropout=0.2),
        Dense(1, activation='sigmoid')
    ])


d:\pythonC\Brainwork Assignments\twitter_sentiment\venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [35]:
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])


model.build(input_shape=(None, max_len))
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 50, 64)         │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 353,089 (1.35 MB)

 Trainable params: 353,089 (1.35 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
X_train_pad.shape
y_train.shape

(1246623,)

In [36]:
history = model.fit(
        X_train_pad, y_train,
        validation_data=(X_test_pad, y_test),
        epochs=3,
        batch_size=64,
        verbose=1
    )

Epoch 1/3
19479/19479 ━━━━━━━━━━━━━━━━━━━━ 3306s 170ms/step - accuracy: 0.5011 - loss: 0.6932 - val_accuracy: 0.5030 - val_loss: 0.6931
Epoch 2/3
19479/19479 ━━━━━━━━━━━━━━━━━━━━ 3306s 170ms/step - accuracy: 0.5011 - loss: 0.6932 - val_accuracy: 0.5030 - val_loss: 0.6931
Epoch 2/3
19479/19479 ━━━━━━━━━━━━━━━━━━━━ 3215s 165ms/step - accuracy: 0.5021 - loss: 0.6932 - val_accuracy: 0.4970 - val_loss: 0.6932
Epoch 3/3
19479/19479 ━━━━━━━━━━━━━━━━━━━━ 3215s 165ms/step - accuracy: 0.5021 - loss: 0.6932 - val_accuracy: 0.4970 - val_loss: 0.6932
Epoch 3/3
19479/19479 ━━━━━━━━━━━━━━━━━━━━ 1724s 88ms/step - accuracy: 0.5018 - loss: 0.6932 - val_accuracy: 0.5030 - val_loss: 0.6931
19479/19479 ━━━━━━━━━━━━━━━━━━━━ 1724s 88ms/step - accuracy: 0.5018 - loss: 0.6932 - val_accuracy: 0.5030 - val_loss: 0.6931


In [37]:
y_pred = np.argmax(model.predict(X_test_pad), axis=1)
print(classification_report(y_test, y_pred, target_names=["Negative","Positive"]))


4870/4870 ━━━━━━━━━━━━━━━━━━━━ 106s 22ms/step
4870/4870 ━━━━━━━━━━━━━━━━━━━━ 106s 22ms/step
              precision    recall  f1-score   support

    Negative       0.50      1.00      0.66     77443
    Positive       0.00      0.00      0.00     78387

    accuracy                           0.50    155830
   macro avg       0.25      0.50      0.33    155830
weighted avg       0.25      0.50      0.33    155830

              precision    recall  f1-score   support

    Negative       0.50      1.00      0.66     77443
    Positive       0.00      0.00      0.00     78387

    accuracy                           0.50    155830
   macro avg       0.25      0.50      0.33    155830
weighted avg       0.25      0.50      0.33    155830



d:\pythonC\Brainwork Assignments\twitter_sentiment\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\pythonC\Brainwork Assignments\twitter_sentiment\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\pythonC\Brainwork Assignments\twitter_sentiment\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(